# EE16A: Homework 4

## Problem 6: Segway Tours

Run the following block of code first to get all the dependencies.

In [ ]:
# %load gauss_elim.py
from gauss_elim import gauss_elim

In [ ]:
from numpy import zeros, cos, sin, arange, around, hstack
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib.patches import Rectangle
import numpy as np
from scipy.interpolate import interp1d
import scipy as sp

## Dynamics

In [ ]:
# Dynamics: state to state
A = np.array([[1, 0.05, -.01, 0],
              [0, 0.22, -.17, -.01],
              [0, 0.1, 1.14, 0.10],
              [0, 1.66, 2.85, 1.14]]);
# Control to state
b = np.array([.01, .21, -.03, -0.44])
nr_states = b.shape[0]

# Initial state
state0 = np.array([-0.3853493, 6.1032227, 0.8120005, -14])

# Final (terminal state)
stateFinal = np.array([0, 0, 0, 0])

## Example of Gaussian Elimination

In [ ]:
# System of example equations: 
# x + y +  5z = 7
# x + 4y - z  = 4
# 3x + y - z  = 4

Q = np.zeros([3,3])

# Matrix construction by specifying column vectors
Q[:,0] = np.array([1, 1, 3]) # coefficients of x
Q[:,1] = np.array([1, 4, 1]) # coefficients of y
Q[:,2] = np.array([5, -1, -1]) # coefficients of z

m = np.array([7, 4, 4])

# Augmented Matrix for system of equations
Q_aug = np.c_[Q, m]

print('Augmented matrix for part f:')
print(Q_aug,'\n')

print('Matrix after Gaussian elimination:')
print(gauss_elim(Q_aug))

## Part (d)

In [ ]:
# Compute the A^2
A2 = np.dot(A,A)

#Compute "Controllability Matrix" Q2 = [Ab b]

Q2 = np.zeros([4,2])
Q2[:,0] = np.dot(A,b)
Q2[:,1] = b

# Augmented Matrix for system of equations

Q2_aug = np.c_[Q2, stateFinal-np.dot(A2,state0)]

print('Augmented matrix for part f:')
print(Q2_aug, '\n')
print('Matrix after Gaussian elimination:')
print(gauss_elim(Q2_aug))

## Part (e)

In [ ]:
# Compute the A^3
A3 = np.dot(A2,A)

#Compute "Controllability Matrix" Q3=[A^2b Ab b]

Q3 = np.zeros([4,3])
Q3[:,0] = np.dot(A2,b)
Q3[:,1] = np.dot(A,b)
Q3[:,2] = b

# Augmented Matrix for system of equations

Q3_aug = np.c_[Q3, stateFinal-np.dot(A3,state0)]

print('Augmented matrix for part g:')
print(Q3_aug, '\n') 
print('Matrix after Gaussian elimination:')
print(gauss_elim(Q3_aug))

## Part (f)

In [ ]:
# Compute the A^4
A4 = np.dot(A3,A)

#Compute "Controllability Matrix" Q4=[A^3b A^2b Ab b]

Q4 = np.zeros([4,4])
Q4[:,0] = np.dot(A3,b)
Q4[:,1] = np.dot(A2,b)
Q4[:,2] = np.dot(A,b)
Q4[:,3] = b

# Augmented Matrix for system of equations

Q4_aug = np.c_[Q4, stateFinal-np.dot(A4,state0)]

print('Augmented matrix for part g:')
print(Q4_aug, '\n') 
print('Matrix after Gaussian elimination:')
print(gauss_elim(Q4_aug), '\n')

# Solution
print('Solution')
print(np.linalg.solve(Q4, stateFinal-np.dot(A4,state0)))


## Part (g)

### Preamble
This function will take care of animating the segway. DO NOT EDIT!

In [ ]:
# frames per second in simulation
fps = 20
# length of the segway arm/stick
stick_length = 1.

def animate_segway(t, states, controls, length):
    #Animates the segway
    
    # Set up the figure, the axis, and the plot elements we want to animate
    fig = plt.figure()
    
    # some config
    segway_width = 0.4
    segway_height = 0.2
    
    # x coordinate of the segway stick
    segwayStick_x = length * np.add(states[:, 0],sin(states[:, 2]))
    segwayStick_y = length * cos(states[:, 2])
    
    # set the limits
    xmin = min(around(states[:, 0].min() - segway_width / 2.0, 1), around(segwayStick_x.min(), 1))
    xmax = max(around(states[:, 0].max() + segway_height / 2.0, 1), around(segwayStick_y.max(), 1))
    
    # create the axes
    ax = plt.axes(xlim=(xmin-.2, xmax+.2), ylim=(-length-.1, length+.1), aspect='equal')
    
    # display the current time
    time_text = ax.text(0.05, 0.9, 'time', transform=ax.transAxes)
    
    # display the current control
    control_text = ax.text(0.05, 0.8, 'control', transform=ax.transAxes)
    
    # create rectangle for the segway
    rect = Rectangle([states[0, 0] - segway_width / 2.0, -segway_height / 2],
        segway_width, segway_height, fill=True, color='gold', ec='blue')
    ax.add_patch(rect)
    
    # blank line for the stick with o for the ends
    stick_line, = ax.plot([], [], lw=2, marker='o', markersize=6, color='blue')

    # vector for the control (force)
    force_vec = ax.quiver([],[],[],[],angles='xy',scale_units='xy',scale=1)

    # initialization function: plot the background of each frame
    def init():
        time_text.set_text('')
        control_text.set_text('')
        rect.set_xy((0.0, 0.0))
        stick_line.set_data([], [])
        return time_text, rect, stick_line, control_text

    # animation function: update the objects
    def animate(i):
        time_text.set_text('time = {:2.2f}'.format(t[i]))
        control_text.set_text('force = {:2.3f}'.format(controls[i]))
        rect.set_xy((states[i, 0] - segway_width / 2.0, -segway_height / 2))
        stick_line.set_data([states[i, 0], segwayStick_x[i]], [0, segwayStick_y[i]])
        return time_text, rect, stick_line, control_text

    # call the animator function
    anim = animation.FuncAnimation(fig, animate, frames=len(t), init_func=init,
            interval=1000/fps, blit=False, repeat=False)
    return anim
    #plt.show()

### Plug in your controller here

In [ ]:
# If you want to try zero control
# controls = np.zeros((4))

controls = np.linalg.solve(Q4, stateFinal-np.dot(A4,state0))
print(controls)

### Simulation
DO NOT EDIT!

In [ ]:
# This will add an extra couple of seconds to the simulation after the input controls with no control
# the effect of this is just to show how the system will continue after the controller "stops controlling"
controls = np.append(controls,[0, 0])

# number of steps in the simulation
nr_steps = controls.shape[0]

# We now compute finer dynamics and control vectors for smoother visualization
Afine = sp.linalg.fractional_matrix_power(A,(1/fps))
Asum = np.eye(nr_states)
for i in range(1, fps):
    Asum = Asum + np.linalg.matrix_power(Afine,i)
    
bfine = np.linalg.inv(Asum).dot(b)

# We also expand the controls in the "intermediate steps" (only for visualization)
controls_final = np.outer(controls, np.ones(fps)).flatten()
controls_final = np.append(controls_final, [0])

# We compute all the states starting from x0 and using the controls
states = np.empty([fps*(nr_steps)+1, nr_states])
states[0,:] = state0;
for stepId in range(1,fps*(nr_steps)+1):
    states[stepId, :] = np.dot(Afine,states[stepId-1, :]) + controls_final[stepId-1] * bfine
    
# Now create the time vector for simulation
t = np.linspace(1/fps,nr_steps,fps*(nr_steps),endpoint=True)
t = np.append([0], t)

### Visualization
DO NOT EDIT!

In [ ]:
%matplotlib nbagg
# %matplotlib qt
anim = animate_segway(t, states, controls_final, stick_length)
anim